In [1]:
import torch
import torch.nn.functional as F
import torchvision.datasets as datasets
import torchvision.transforms as transforms
from torch.utils.data import Dataset
%matplotlib inline

from pclib.nn.models import FCClassifierUs
from pclib.optim.train import train
from pclib.optim.eval import track_vfe, accuracy
from pclib.utils.customdataset import PreloadedDataset

In [2]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
device

device(type='cuda', index=0)

In [3]:
words = open('../Datasets/names.txt', 'r').read().splitlines()
chars = sorted(list(set(''.join(words))))
stoi = {c: i+1 for i, c in enumerate(chars)}
stoi['.'] = 0
itos = {i: c for c, i in stoi.items()}
vocab_len = len(stoi)
print(vocab_len)

27


In [7]:
BLOCK_SIZE = 5

def BuildDataset(words, device=device):
    X, Y = [], []
    for w in words:
        context = [0] * BLOCK_SIZE
        for ch in w:
            idx = stoi[ch]
            X.append(context)
            Y.append(idx)
            context = context[1:] + [idx]
    return torch.tensor(X, dtype=torch.float32).to(device), torch.tensor(Y).to(device)

class CustomDataset(Dataset):
    def __init__(self, X, Y):
        self.X = X
        self.Y = Y
    def __len__(self):
        return self.X.shape[0]
    def __getitem__(self, idx):
        return self.X[idx], self.Y[idx]
    def apply_transform(self):
        pass

split1 = int(len(words) * 0.8)
split2 = int(len(words) * 0.9)


import random
seed = 42
random.seed(seed)
random.shuffle(words)

Xs, Ys = BuildDataset(words, device)
train_dataset = CustomDataset(Xs[:split1], Ys[:split1])
val_dataset = CustomDataset(Xs[split1:split2], Ys[split1:split2])
test_dataset = CustomDataset(Xs[split2:], Ys[split2:])

In [8]:
INPUT_SHAPE = BLOCK_SIZE
NUM_CLASSES = vocab_len
torch.manual_seed(42)

model_name = 'FCClUs-BS5'
model = FCClassifierUs(
    in_features = INPUT_SHAPE, 
    num_classes = NUM_CLASSES,
    hidden_sizes = [200, 200],
    bias=True, 
    symmetric=True, 
    precision_weighted=False,
    actv_fn=F.tanh,
    steps=100,
    gamma=0.34,
    ).to(device)

In [9]:
NUM_EPOCHS = 10
BATCH_SIZE = 256

log_dir = f'examples/names/logs/{model_name}'

train(
    model, 
    train_dataset, 
    val_dataset, 
    NUM_EPOCHS, 
    lr=0.001,
    c_lr=0.0,
    batch_size=BATCH_SIZE,
    reg_coeff=0.02,
    optim='AdamW',
    save_best=False,
    log_dir=log_dir,
)
NUM_EPOCHS = 5
train(
    model, 
    train_dataset, 
    val_dataset, 
    NUM_EPOCHS, 
    lr=0.0001,
    c_lr=0.01,
    batch_size=BATCH_SIZE,
    reg_coeff=0.02,
    optim='AdamW',
    save_best=False,
    log_dir=log_dir,
)